In [1]:
import numpy as np
import pandas as pd
import random
from collections import defaultdict

In [2]:
edges = pd.read_csv('data/musae_git_edges.csv')
edges

,id_1,id_2
0,0,23977
1,1,34526
2,1,2370
3,1,14683
4,1,29982
...,...,...
288998,37527,37596
288999,37529,37601
289000,37644,2347
289001,25879,2347


In [3]:
class edges:
    def __init__(self):
        self.edge_list = set()
        self.neighbour_list = {}
        self.n_edges = 0
        
    def insert(self, u, v):
        if u in self.neighbour_list.keys():
            self.neighbour_list[u].append(v)
        else:
            self.neighbour_list[u] = [v]
        
        if v in self.neighbour_list.keys():
            self.neighbour_list[v].append(u)
        else:
            self.neighbour_list[v] = [u]
        
        self.edge_list.add((u,v))
        self.n_edges += 1
        
    def delete(self, u, v):
        self.neighbour_list[u].remove(v)
        neighbour_u = self.neighbour_list[u]
        if len(neighbour_u) == 0:
            del self.neighbour_list[u]

        self.neighbour_list[v].remove(u)
        neighbour_v = self.neighbour_list[v]
        if len(neighbour_v) == 0:
            del self.neighbour_list[v]

        self.edge_list.remove((u,v))
        self.n_edges -= 1
        
    def get_neighbours(self,u):
        return self.neighbour_list[u]

    def get_n_edges(self):
        return self.n_edges

    def get_n_vertices(self):
        return len(self.neighbour_list)

    def get_vertices(self):
        return self.neighbour_list.keys()

    def get_edges(self):
        return list(self.edge_list)

In [4]:
class triestBase:
    def __init__(self, memory):
        self.memory = memory
        self.S = edges()
        self.global_counter = 0
        self.local_counter = defaultdict(lambda:0)
        
    def fit(self, graph_streaming):
        t = 0
        with open(graph_streaming) as f:
            next(f)
            lines = f.read().splitlines()
            for line in lines:
                u, v = sorted([int(e) for e in line.split(',')])
                t += 1
                if self.sample(t, u, v):
                    self.S.insert(u, v)
                    self.update_counter('+', u, v)
                '''print("t: {}".format(t))#---------------------------------------------------------------------------------
                print("list edges: {}".format(self.S.edge_list))
                print("total edges: {}".format(self.S.n_edges))
                print("all neighrbours: {}".format(self.S.neighbour_list))
                print("global T: {}".format(self.global_counter))
                print("local T: {}".format(self.local_counter))'''
                    
        global_triangles = self.estimate_global_counter(t)
        
        return int(global_triangles)
                
    def sample(self, t, u, v):
        if t <= self.memory:
            return True
        else:
            if self.flip_coin(t):
                n_edges = self.S.get_n_edges()
                index = random.randint(0, n_edges-1)
                u, v = random.sample(self.S.edge_list, 1)[0]
                self.S.delete(u,v)
                self.update_counter('-', u, v)
                return True
            else:
                return False
            
    def flip_coin(self, t):
        p_head = self.memory/t
        p = random.random()
        if p <= p_head:
            return True
        else:
            return False
        
    def update_counter(self, operation, u, v):
        vertices = self.S.get_vertices()
        if (u not in vertices) or (v not in vertices):
            return
        u_neighbours = self.S.get_neighbours(u)
        v_neighbours = self.S.get_neighbours(v)
        
        shared_neighbours = list(set(u_neighbours) & set(v_neighbours))
        n_shared_neighbours = len(shared_neighbours)
        
        if n_shared_neighbours == 0:
            return
        
        if operation == '+':
            self.global_counter +=  n_shared_neighbours
            self.local_counter[u] +=  n_shared_neighbours
            self.local_counter[v] +=  n_shared_neighbours

            for c in shared_neighbours:
                self.local_counter[c] += 1
        
        if operation == '-':
            self.global_counter -= n_shared_neighbours

            self.local_counter[u] -= n_shared_neighbours
            if self.local_counter[u] == 0:
                del self.local_counter[u]

            self.local_counter[v] -= n_shared_neighbours
            if self.local_counter[v] == 0:
                del self.local_counter[v]

            for c in shared_neighbours:
                self.local_counter[c] -= 1
            if self.local_counter[c] == 0:
                del self.local_counter[c]
                
    def estimate_global_counter(self, t):
        M = self.memory
        estimator = np.max([1, (t*(t-1)*(t-2))/(M*(M-1)*(M-2))])
        return estimator * self.global_counter

In [5]:
class triestImprove:
    def __init__(self, memory):
        self.memory = memory
        self.S = edges()
        self.global_counter = 0
        self.local_counter = defaultdict(lambda:0)
        self.t = 0
        
    def fit(self, graph_streaming):
        #t = 0
        with open(graph_streaming) as f:
            next(f)
            lines = f.read().splitlines()
            for line in lines:
                u, v = sorted([int(e) for e in line.split(',')])
                self.t += 1
                #changes #1: moving update_counter before if block 
                self.update_counter('+', u, v)
                if self.sample(self.t, u, v):
                    self.S.insert(u, v)
                    #self.update_counter('+', u, v)
                '''print("t: {}".format(self.t))#---------------------------------------------------------------------------------
                print("list edges: {}".format(self.S.edge_list))
                print("total edges: {}".format(self.S.n_edges))
                print("all neighrbours: {}".format(self.S.neighbour_list))
                print("global T: {}".format(self.global_counter))
                print("local T: {}".format(self.local_counter))'''
                    
        #global_triangles = self.estimate_global_counter(t)
        global_triangles = self.global_counter
        
        return float(global_triangles)
                
    def sample(self, t, u, v):
        if self.t <= self.memory:
            return True
        else:
            if self.flip_coin(t):
                n_edges = self.S.get_n_edges()
                index = random.randint(0, n_edges-1)
                u, v = random.sample(self.S.edge_list, 1)[0]
                self.S.delete(u,v)
                #changes #2: removing update_counter when an edge is removed from S 
                #self.update_counter('-', u, v)
                return True
            else:
                return False
            
    def flip_coin(self, t):
        p_head = self.memory/t
        p = random.random()
        if p <= p_head:
            return True
        else:
            return False
        
    def update_counter(self, operation, u, v):
        vertices = self.S.get_vertices()
        if (u not in vertices) or (v not in vertices):
            return
        u_neighbours = self.S.get_neighbours(u)
        v_neighbours = self.S.get_neighbours(v)
        
        shared_neighbours = list(set(u_neighbours) & set(v_neighbours))
        n_shared_neighbours = len(shared_neighbours)
        
        if n_shared_neighbours == 0:
            return
        
        if operation == '+':
            self.global_counter +=  n_shared_neighbours
            self.local_counter[u] +=  n_shared_neighbours
            self.local_counter[v] +=  n_shared_neighbours

            for c in shared_neighbours:
                #changes #3: performs a weighted increase of the counters 
                #self.local_counter[c] += 1
                self.local_counter[c] += int(max(1, ((self.t - 1) * (self.t - 2)) / (self.memory * (self.memory - 1))))
        
        '''if operation == '-':
            self.global_counter -= n_shared_neighbours

            self.local_counter[u] -= n_shared_neighbours
            if self.local_counter[u] == 0:
                del self.local_counter[u]

            self.local_counter[v] -= n_shared_neighbours
            if self.local_counter[v] == 0:
                del self.local_counter[v]

            for c in shared_neighbours:
                self.local_counter[c] -= 1
            if self.local_counter[c] == 0:
                del self.local_counter[c]'''
                
    '''def estimate_global_counter(self, t):
        M = self.memory
        estimator = np.max([1, (t*(t-1)*(t-2))/(M*(M-1)*(M-2))])
        return estimator * self.global_counter'''

In [6]:
if __name__ == '__main__':
    random.seed(333)
    filename = 'data/dummy.csv'
    
    graph_streaming = triestImprove(7)
    global_triangles = graph_streaming.fit(filename)
    
    global_triangles

In [7]:
global_triangles

6.0

In [8]:
if __name__ == '__main__':
    random.seed(333)
    filename = 'data/dummy.csv'
    
    graph_streaming = triestBase(7)
    global_triangles = graph_streaming.fit(filename)
    
    global_triangles

In [9]:
global_triangles

6

In [10]:
if __name__ == '__main__':
    random.seed(333)
    filename = 'data/musae_git_edges.csv'
    
    graph_streaming = triestImprove(10000)
    global_triangles = graph_streaming.fit(filename)

In [11]:
# M = 10000
global_triangles

5340.0

In [12]:
if __name__ == '__main__':
    random.seed(333)
    filename = 'data/musae_git_edges.csv'
    
    graph_streaming = triestBase(10000)
    global_triangles = graph_streaming.fit(filename)

In [13]:
# M = 10000
global_triangles

362179

In [14]:
if __name__ == '__main__':
    random.seed(333)
    filename = 'data/musae_git_edges.csv'
    
    graph_streaming = triestImprove(100000)
    global_triangles = graph_streaming.fit(filename)

In [15]:
# M = 100000
global_triangles

184877.0

In [16]:
if __name__ == '__main__':
    random.seed(333)
    filename = 'data/musae_git_edges.csv'
    
    graph_streaming = triestBase(100000)
    global_triangles = graph_streaming.fit(filename)

In [17]:
# M = 100000
global_triangles

519128

In [18]:
if __name__ == '__main__':
    random.seed(333)
    filename = 'data/musae_git_edges.csv'
    
    graph_streaming = triestImprove(289003)
    global_triangles = graph_streaming.fit(filename)

In [19]:
# Ground truth
global_triangles

523810.0

In [20]:
if __name__ == '__main__':
    random.seed(333)
    filename = 'data/musae_git_edges.csv'
    
    graph_streaming = triestBase(289003)
    global_triangles = graph_streaming.fit(filename)

In [21]:
# Ground truth
global_triangles

523810